In [ ]:
%pip install gym-trading-env pandas numpy matplotlib stable_baselines3 'shimmy>=0.2.1'

In [1]:
import gym_trading_env

import gymnasium as gym
import pandas as pd
from matplotlib import pyplot as plt

from stable_baselines3 import DQN,A2C
from stable_baselines3.common.vec_env import DummyVecEnv

2023-07-21 21:14:59.969991: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-21 21:14:59.999053: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-21 21:15:00.545474: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [4]:
df = pd.read_csv('Data/NIFTY50.csv')
df['DatetimeIndex'] = pd.to_datetime(df['DatetimeIndex'])
df.set_index('DatetimeIndex', inplace=True)
# df["feature_pct_change"] = df["close"].pct_change()
# df["feature_high"] = df["high"] / df["close"] - 1
# df["feature_low"] = df["low"] / df["close"] - 1
# df.dropna(inplace= True)

df.head()

,open,high,low,close
DatetimeIndex,,,,
2013-01-22,6080.15,6101.30,6040.50,6048.50
2013-01-23,6052.85,6069.80,6021.15,6054.30
2013-01-24,6046.20,6065.30,6007.85,6019.35
2013-01-25,6024.50,6080.55,6014.45,6074.65
2013-01-28,6082.10,6088.40,6061.40,6074.80


In [5]:
env=gym.make('TradingEnv',df=df,verbose=1,name='NIFTY50',trading_fees = 0.01/100,borrow_interest_rate= 0.0003/100,windows=5)
sum=0
done, truncated = False, False
observation, info = env.reset()
while not done and not truncated:
    # Pick a position by its index in your position list (=[-1, 0, 1])....usually something like : position_index = your_policy(observation)
    position_index = env.action_space.sample() # At every timestep, pick a random position index from your position list (=[-1, 0, 1])
    observation, reward, done, truncated, info = env.step(position_index)
    sum+=reward
    # if reward>0 :
    #     sum+=reward
    # else: 
    #     sum-=reward
print(sum)

# env.save_for_render
# env.unwrapped.save_for_render(dir = "render_logs")

Market Return : 94.06%   |   Portfolio Return : 51.59%   |   
0.41602610769378656


In [28]:
def env_maker():
    return gym.make('TradingEnv',df=df,verbose=1,name='NIFTY50',trading_fees = 0.01/100,borrow_interest_rate= 0.0003/100)

# Create the environment using make_vec_env
env = DummyVecEnv([env_maker])

# Train the A2C model
model = A2C('MlpPolicy', env, verbose=1)
model.learn(total_timesteps=1000)

Using cpu device
------------------------------------
| time/                 |          |
|    fps                | 1028     |
|    iterations         | 100      |
|    time_elapsed       | 0        |
|    total_timesteps    | 500      |
| train/                |          |
|    entropy_loss       | -0.693   |
|    explained_variance | -214     |
|    learning_rate      | 0.0007   |
|    n_updates          | 99       |
|    policy_loss        | 0.0828   |
|    value_loss         | 0.0185   |
------------------------------------
------------------------------------
| time/                 |          |
|    fps                | 914      |
|    iterations         | 200      |
|    time_elapsed       | 1        |
|    total_timesteps    | 1000     |
| train/                |          |
|    entropy_loss       | -0.692   |
|    explained_variance | -1.42    |
|    learning_rate      | 0.0007   |
|    n_updates          | 199      |
|    policy_loss        | 0.00958  |
|    value_loss      

In [29]:
df_test=pd.read_csv('Data/test_data.csv')
df_test['DatetimeIndex'] = pd.to_datetime(df_test['DatetimeIndex'])
df_test.set_index('DatetimeIndex', inplace=True)

In [44]:
test_env=gym.make('TradingEnv',df=df_test,verbose=1,name='NIFTY50',trading_fees = 0.01/100,borrow_interest_rate= 0.0003/100)
state,_=test_env.reset()
truncated=0
terminated=0
while True:
    action,_states=model.predict(state)
    n_state,reward,truncated,terminated,info=test_env.step(action)
    if truncated or terminated:
        print("info", info)
        break


Market Return : 18.91%   |   Portfolio Return : 13.45%   |   
info {'idx': 247, 'step': 247, 'date': numpy.datetime64('2023-07-21T00:00:00.000000000'), 'position_index': array(1), 'position': 1, 'real_position': 1.0, 'data_high': 19887.4, 'data_close': 19745.0, 'data_low': 19700.0, 'data_open': 19800.45, 'portfolio_valuation': 1134.4643899994128, 'portfolio_distribution_asset': 0.057455780703945955, 'portfolio_distribution_fiat': 0, 'portfolio_distribution_borrowed_asset': 0, 'portfolio_distribution_borrowed_fiat': 0, 'portfolio_distribution_interest_asset': 0.0, 'portfolio_distribution_interest_fiat': 0.0, 'reward': -0.011788935034292816}
